import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [2]:
spark = SparkSession.builder.master("local").appName("PySpark_Basics").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/05 10:11:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


read csv with inferschema

In [3]:
df = spark.read.format("csv")\
    .option('header', 'true')\
    .option("interschema", 'true')\
    .load('ds_salaries.csv')
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: string (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: string (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [4]:
df = spark.read.format("csv")\
    .option('header', 'true')\
    .option("interschema", 'true')\
    .load('ds_salaries.csv')
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: string (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: string (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



write schema of scv on screen

In [14]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [15]:
schema = StructType([
StructField('_c0', IntegerType()),
    StructField('work_year', IntegerType()),
    StructField('experience_level', StringType()),
    StructField('employment_type', StringType()),
    StructField('job_title', StringType()),
    StructField('salary', IntegerType()),
    StructField('salary_currency', StringType()),
    StructField('salary_in_usd', IntegerType()),
    StructField('employee_residence', StringType()),
    StructField('remote_ratio', IntegerType()),
    StructField('company_location', StringType()),
    StructField('company_size', StringType())
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [16]:
df2 = spark.read.format("csv")\
    .option('header', 'true')\
    .option('schema', schema)\
    .load("ds_salaries.csv")

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [17]:
df2.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: string (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: string (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

In [18]:
df = df.withColumnRenamed("_c0", "id")

print data in dataframe using df.show

In [19]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

24/01/05 10:11:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///Users/germanvaschelin/Desktop/Программы/Task4/PySpark_Basics/ds_salaries.csv


In [20]:
df = df.withColumn("salary_in_usd", col("salary_in_usd").cast(IntegerType()))
df = df.withColumn("salary", col("salary").cast(IntegerType()))
df = df.withColumn("remote_ratio", col("remote_ratio").cast(IntegerType()))
df = df.withColumn("id", col("id").cast(IntegerType()))

In [21]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



print data in dataframe using display(df.toPandas())

In [22]:
import pandas
df.toPandas()

ModuleNotFoundError: No module named 'distutils'

create df_job_title that consists from all job_titles without duplicates

In [23]:
df_job_title = df.select(col("job_title")).distinct()

print all rows from df_job_titles without truncating jobs

In [24]:
df_job_title.show(df_job_title.count(), truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
|Research S

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [25]:
df_analytic = df.groupby("job_title")\
    .agg(max("salary_in_usd").alias('max_salary'),
         avg("salary_in_usd").alias('avg_salary'),
         min("salary_in_usd").alias('min_salary'))

print all rows from df_analytic without trancating jobs

In [26]:
df_analytic.show(df_analytic.count(), truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|Lead Data Engineer                      |276000    |139724.5          |56000     |
|Head of Machine Learning                |79039     |79039.0           |79039     |
|Data Specialist                         |165000    |165000.0          |165000    |
|Data Analytics Lead                     |405000    |405000.0          |405000    |
|Machine Learning Scientist              |260000    |158412.5          |12000     |
|Lead Data Analyst                       |170000    |92203.0           |19609     |
|Data Engineering Manager                |174000    |123227.2          |59303     |
|Staff Data Scientist                    |105000    |105000.0          |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [27]:
df_analytic = df_analytic.withColumn('row_id', row_number()\
    .over(Window.orderBy(col('avg_salary').desc())))

print all data from df_analytic

In [28]:
df_analytic.show(df_analytic.count(), truncate=False)

24/01/05 10:12:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------------------------------+----------+------------------+----------+------+
|job_title                               |max_salary|avg_salary        |min_salary|row_id|
+----------------------------------------+----------+------------------+----------+------+
|Data Analytics Lead                     |405000    |405000.0          |405000    |1     |
|Principal Data Engineer                 |600000    |328333.3333333333 |185000    |2     |
|Financial Data Analyst                  |450000    |275000.0          |100000    |3     |
|Principal Data Scientist                |416000    |215242.42857142858|148261    |4     |
|Director of Data Science                |325000    |195074.0          |130026    |5     |
|Data Architect                          |266400    |177873.9090909091 |90700     |6     |
|Applied Data Scientist                  |380000    |175655.0          |54238     |7     |
|Analytics Engineer                      |205300    |175000.0          |135000    |8     |

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [29]:
df_analytic = df_analytic.select("row_id", "job_title", "max_salary", "avg_salary", "min_salary")

print df_analytic now

In [30]:
df_analytic.show(df_analytic.count(), truncate=False)

24/01/05 10:12:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/05 10:12:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+----------------------------------------+----------+------------------+----------+
|row_id|job_title                               |max_salary|avg_salary        |min_salary|
+------+----------------------------------------+----------+------------------+----------+
|1     |Data Analytics Lead                     |405000    |405000.0          |405000    |
|2     |Principal Data Engineer                 |600000    |328333.3333333333 |185000    |
|3     |Financial Data Analyst                  |450000    |275000.0          |100000    |
|4     |Principal Data Scientist                |416000    |215242.42857142858|148261    |
|5     |Director of Data Science                |325000    |195074.0          |130026    |
|6     |Data Architect                          |266400    |177873.9090909091 |90700     |
|7     |Applied Data Scientist                  |380000    |175655.0          |54238     |
|8     |Analytics Engineer                      |205300    |175000.0          |135000    |

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [31]:
df_exp_lvl = df.groupBy("experience_level").agg(max("salary_in_usd").alias("biggest_salary"))

print here df_exp_lvl

In [32]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



In [33]:
df_exp_lvl.show(df_exp_lvl.count(), truncate=False)

+----------------+--------------+
|experience_level|biggest_salary|
+----------------+--------------+
|EX              |600000        |
|MI              |450000        |
|EN              |250000        |
|SE              |412000        |
+----------------+--------------+



create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [34]:
df_best = df.alias("a").join(df_exp_lvl.alias("b"), 
                  [df.salary_in_usd == df_exp_lvl.biggest_salary, 
                   df.experience_level == df_exp_lvl.experience_level],
                  'inner')\
                      .select(col("a.id"), col("a.experience_level"), col("b.biggest_salary"), col("a.employee_residence"))

print df_best

In [35]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 33|              MI|        450000|                US|
| 37|              EN|        250000|                US|
| 63|              SE|        412000|                US|
| 97|              MI|        450000|                US|
|252|              EX|        600000|                US|
+---+----------------+--------------+------------------+



24/01/05 10:13:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: _c0, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///Users/germanvaschelin/Desktop/Программы/Task4/PySpark_Basics/ds_salaries.csv


drop duplicates if exist by experience_level

In [36]:
df_best = df_best.dropDuplicates(['experience_level'])

print df_best

In [37]:
df_best.show()

24/01/05 10:13:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: _c0, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///Users/germanvaschelin/Desktop/Программы/Task4/PySpark_Basics/ds_salaries.csv


+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [38]:
df_new_best = df_best.select('experience_level', 'biggest_salary', 'employee_residence',
                            when(col('experience_level') == 'MI', 'Middle')\
                            .when(col('experience_level') == 'SE', 'Senior')\
                            .otherwise('Null').alias('new_experience_level'))

print df_new_best

In [39]:
df_new_best.show()

+----------------+--------------+------------------+--------------------+
|experience_level|biggest_salary|employee_residence|new_experience_level|
+----------------+--------------+------------------+--------------------+
|              EN|        250000|                US|                Null|
|              EX|        600000|                US|                Null|
|              MI|        450000|                US|              Middle|
|              SE|        412000|                US|              Senior|
+----------------+--------------+------------------+--------------------+



write df_new_best like 1.csv and load then it to df_final

In [40]:
df_new_best.write.mode('overwrite').options(header = True, delimiter = ',').csv('1.csv')
df_final = spark.read.format('csv')\
    .options(header = True, interschema = True)\
    .load('1.csv')

print df_final

In [41]:
df_final.show()

+----------------+--------------+------------------+--------------------+
|experience_level|biggest_salary|employee_residence|new_experience_level|
+----------------+--------------+------------------+--------------------+
|              EN|        250000|                US|                Null|
|              EX|        600000|                US|                Null|
|              MI|        450000|                US|              Middle|
|              SE|        412000|                US|              Senior|
+----------------+--------------+------------------+--------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [42]:
df_final = df_final.filter(col('new_experience_level')!='Null')\
    .join(df, 
        [df.salary_in_usd == df_final.biggest_salary, df.employee_residence == df_final.employee_residence])

print df_final

In [43]:
df_final.show(truncate=False)

+----------------+--------------+------------------+--------------------+---+---------+----------------+---------------+----------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|experience_level|biggest_salary|employee_residence|new_experience_level|id |work_year|experience_level|employment_type|job_title             |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+----------------+--------------+------------------+--------------------+---+---------+----------------+---------------+----------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|MI              |450000        |US                |Middle              |33 |2020     |MI              |FT             |Research Scientist    |450000|USD            |450000       |US                |0           |US              |M           |
|SE              |412000    

24/01/05 10:13:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///Users/germanvaschelin/Desktop/Программы/Task4/PySpark_Basics/ds_salaries.csv


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [46]:
variable = df_final.select('biggest_salary','salary_in_usd').agg(max(col('salary_in_usd')).alias('biggest')).drop('salary_in_usd')
variable.show()

+-------+
|biggest|
+-------+
| 450000|
+-------+



It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system